# 生成图的邻接矩阵占用内存情况

### 导入相关库 

In [1]:
#!/usr/bin/python2.7
# -*- coding:utf-8 -*-
import numpy as np
from scipy.spatial import distance as ds
import datetime
from NHKlib2 import *
from sklearn.decomposition import KernelPCA,PCA
import scipy.io as sio
import os,sys,gc,psutil
import scipy as sp

process = psutil.Process(os.getpid())

### 导入数据

In [2]:
matrix = sio.loadmat('matrix.mat')['matrix']

In [3]:
window = input("Please enter the size of window: ")

Please enter the size of window: 25


### 生成一个图的邻接矩阵，并比较原始矩阵与稀疏矩阵的内存使用情况

In [4]:
temp = []
temp = matrix[0:window,:]
# 以列向量为变量，计算各个变量间的协方差矩阵cov
cov = np.cov(temp.T)
cov[range(len(cov)), range(len(cov))] = 0
sortcov = np.array(cov).reshape((cov.size, 1))
sortcov.sort(0)
# 取协方差为前10%的作为连通图
index = sortcov[int(len(sortcov) * 0.9)]
mem_1 = process.memory_info().rss
if index == 0:
    graph = np.where(cov > index, 1, 0).astype(int)
else:
    graph = np.where(cov >= index, 1, 0).astype(int)
mem_2 = process.memory_info().rss
g1 = sp.sparse.csc_matrix(graph)
mem_3 = process.memory_info().rss
osize = mem_2 - mem_1
ssize = mem_3 - mem_2
print "original size is %fMB, sparse matrix size is %fMB"%(osize/1048576.0,ssize/1048576.0)

original size is 30.953125MB, sparse matrix size is 6.199219MB


可见生成全部6194个图的邻接矩阵会造成内存溢出,即使使用稀疏矩阵存储也至少需要37.5GB的内存